In [8]:
from decaf.scripts.imagenet import DecafNet
from os import path
from skimage import color, img_as_ubyte
import cPickle

class AVIClassifier(object):
    def __init__(self, decaf_folder = None, classifer_file = None):
        if decaf_folder is None:
            decaf_folder = '../models/imagenet_pretrained/'
        if classifer_file is None:
            classifer_file = "lg_classifier_public"
        self.net = DecafNet(path.join(decaf_folder, 'imagenet.decafnet.epoch90'),
                            path.join(decaf_folder, 'imagenet.decafnet.meta'))
        self.feat_layer = 'fc6_cudanet_out'
        self.classifier = cPickle.load(open(classifer_file, "r"))
    def predict(self, img):
        im = img_as_ubyte(color.rgb2gray(img))
        scores = self.net.classify(im, center_only = True)
        feats = self.net.feature(self.feat_layer).flatten()
        defect_probs = self.classifier.predict_proba(feats)
        return sorted(zip(self.classifier.classes_, defect_probs[0]), key = lambda (cls, prob): prob, reverse=True)

#example

In [12]:
import matplotlib.pylab as plt
%matplotlib inline
import numpy as np
from skimage import io
im = io.imread("../data/NEU surface defect database/Cr_88.bmp")
from sklearn.feature_extraction.image import extract_patches_2d
patches = extract_patches_2d(im, (190,190),max_patches = 2)
ims =[]
pred_y = []
for patch in patches:
    avi = AVIClassifier()
    pred = avi.predict(patch)
    ims.append(patch)
    pred_y.append(pred)
    
print pred_y[0]

# #plot results
# io.imshow(im)
# plt.figure(figsize = (20,50))
# for i in xrange(50):
#     prob = [l[1] for l in pred_y[i]]
#     plt.subplot(10,5,i)
#     io.imshow(ims[i])
#     plt.title(label_names[np.argmax(prob)] + str(np.max(prob)))

[('Cr', 0.99997097381777988), ('PS', 2.356690297291142e-05), ('RS', 4.8728099740662068e-06), ('Pa', 5.6601305618980938e-07), ('Sc', 2.0456215259766887e-08), ('In', 1.5966755113122746e-15)]


(1, 6)